In [4]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os
import requests
import json
from urllib.parse import urlencode, urlparse, parse_qsl
import pickle

# Explore Google API, let's see what we find

In [ ]:
db

In [6]:
# We are importing our API key
load_dotenv()

True

In [7]:
g_key = os.getenv("GOOGLE_KEY2")

In [36]:
# Our first dive into Google API looking for gluten free
data_type = "json"
endpoint = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/{data_type}"
params = {
    
    "input":"gluten-free",
    "inputtype":"textquery",
    "fields":"formatted_address,name,geometry",
    "locationbias":"point:lat,lng",
    "key": f"{g_key}"
}

url_params = urlencode(params)
url = f"{endpoint}?{url_params}"

In [37]:
data = requests.get(url)

In [38]:
# We have a postive response
data

<Response [200]>

In [39]:
type(data)

requests.models.Response

In [40]:
data = data.json()

In [41]:
type(data)

dict

In [42]:
# I suppose that because we weren't really specific we didn't get much of a results
# but at least we know it works
print(data)

{'candidates': [{'formatted_address': 'Calle de Hortaleza, 3, 28004 Madrid, España', 'geometry': {'location': {'lat': 40.420441, 'lng': -3.701066}, 'viewport': {'northeast': {'lat': 40.42177422989272, 'lng': -3.699668270107278}, 'southwest': {'lat': 40.41907457010728, 'lng': -3.702367929892722}}}, 'name': 'Celicioso'}], 'status': 'OK'}


In [9]:
# Let's get serious here and create a funtcion that we can reuse
def nearby_search(lat, long, radius_in_meters, keyword):
    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {

        "location":f"{lat},{long}",
        "radius":f"{radius_in_meters}",
        #"rankby" : "<distance - must include keyword,name,or type",
        "keyword":f"{keyword}",
        "inputtype":"textquery",
        "fields":"formatted_address,name,geometry",
        "key": f"{g_key}"
    }

    
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    data = requests.get(url)
    return data.json()

In [10]:
gluten_free = nearby_search(40.420011, -3.701842, 10_000, "gluten-free")

In [11]:
goog_gf = gluten_free["results"]

In [12]:
# Oh....it's like getting Christmas presents!!! 
len(goog_gf)

20

In [13]:
# Let's open them and see what we got!
goog_gf[0]

{'business_status': 'CLOSED_TEMPORARILY',
 'geometry': {'location': {'lat': 40.4340759, 'lng': -3.7085295},
  'viewport': {'northeast': {'lat': 40.43542857989272,
    'lng': -3.707249820107279},
   'southwest': {'lat': 40.43272892010728, 'lng': -3.709949479892722}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'name': 'Kint',
 'permanently_closed': True,
 'photos': [{'height': 4608,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108913216153539893740">Laura Garmo</a>'],
   'photo_reference': 'ATtYBwJSUiBIyM3Yf2mWoVs0K1wS6c7A5VS-42KkpN3_VE4ynnOk08TeavlVAeVQZGg0BuYjKb_JZaxJ4NWInAnYlDQGVr2loI4_K4cdkPE_8KmnB8DTQ9TKEudpOLvu6q5VPzWeQzG8GpUglFMjZHQce5jtFe2glvM-TN9lNADeqgDNNWnd',
   'width': 3456}],
 'place_id': 'ChIJS1Nr5l0oQg0RuXCYXOa0yFc',
 'plus_code': {'compound_code': 'C7MR+JH Madrid',
  'global_code': '8CGRC7MR+JH'},
 'rating': 4.2,
 'reference': 'ChIJS1Nr5l0oQg0RuXCYXOa0yFc',
 'scope': 'GOOGLE',
 'types': ['restaur

In [14]:
pickle_out = open("/Users/claytonlouden/projects/final_project/pickled/goog_gf.pickle","wb")
pickle.dump(goog_gf, pickle_out)
pickle_out.close()